In [2]:
from tensorflow import keras
from HelperFunctions.FileReader import FileReader
from HelperFunctions.OptimzationClass import OptmiziationClass
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# loading pre-trained tensorflow models 
lstm_d50 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_ D_v50_test.h5')
lstm_torque = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_torque_test.h5')
lstm_zone2 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 2_test.h5')
lstm_zone3 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 3_test.h5')
lstm_zone4 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 4_test.h5')
lstm_zone5 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 5_test.h5')
lstm_zone6 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 6_test.h5')
lstm_zone7 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 7_test.h5')
lstm_zone8 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 8_test.h5')